In [1]:
from transformers import pipeline
model_sentiment = pipeline("sentiment-analysis", model="climatebert/distilroberta-base-climate-tcfd",device=0)
#model_sentiment = pipeline("sentiment-analysis",device=0)

In [6]:
import pandas as pd
import numpy as np
df = pd.read_parquet("test-tcdf.parquet")

In [ ]:
# load the csv file into a df2
df2 = pd.read_csv("test-tcdf.csv")

In [7]:
# df unique values label
df['label'].unique()

array([2, 3, 0, 4, 1], dtype=int64)

In [8]:
# create dictiony that maps the following governance, strategy, risk management, metrics and targets in numbers
# 0: governance, 1: strategy, 2: risk management, 3: metrics and targets
mappint_dict = {'governance': 4, 'strategy': 2, 'risk': 3, 'metrics':1, 'none':0}


In [9]:
df.groupby('label').count()

,text
label,
0,80
1,49
2,197
3,48
4,26


In [10]:
df = df[df['label'] != 0]

In [11]:
ctext = "text"
sent_col = "prediction"

In [12]:
model_sentiment("none")

[{'label': 'strategy', 'score': 0.36417022347450256}]

In [36]:
df[sent_col] = "none"
df[sent_col+"_VALUE"] = 0
i = 0
sent = []
s_val = []
for s in df[ctext].to_list():
    #print(s)
    i+=1
    if i%100 ==0:
        print(i,end=" ")
    if len(s)>1:
        if(len(s)>2000):
            s=s[:2000]
        result = model_sentiment(s)[0]
        # map the result['label'] into numbers using the mapping_dict dictionary, if the value is not in the dictionary, then it is mapped to 0
        rez = mappint_dict.get(result['label'].lower(),0)
        sent.append(rez)
        s_val.append(-result['score'])
    else:
        sent.append(0)
        s_val.append(0)
df[sent_col] = sent
df[sent_col+"_VALUE"] = s_val

C:\Users\dimit\AppData\Local\Temp\ipykernel_24720\1921190832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sent_col] = "none"
C:\Users\dimit\AppData\Local\Temp\ipykernel_24720\1921190832.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sent_col+"_VALUE"] = 0
c:\ProgramData\Anaconda3\envs\trans\lib\site-packages\transformers\pipelines\base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


100 200 300 

C:\Users\dimit\AppData\Local\Temp\ipykernel_24720\1921190832.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sent_col] = sent
C:\Users\dimit\AppData\Local\Temp\ipykernel_24720\1921190832.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sent_col+"_VALUE"] = s_val


In [38]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))


(0.7049889543446244, 0.7176506755572202, 0.7096307121882569, None)
[[ 36  10   3   0]
 [ 17 166  10   4]
 [  2  14  27   5]
 [  1   4   2  19]]
              precision    recall  f1-score   support

           1       0.64      0.73      0.69        49
           2       0.86      0.84      0.85       197
           3       0.64      0.56      0.60        48
           4       0.68      0.73      0.70        26

    accuracy                           0.78       320
   macro avg       0.70      0.72      0.71       320
weighted avg       0.78      0.78      0.77       320



In [39]:
df

,text,label,prediction,prediction_VALUE
0,Sustainable strategy ‘red lines’ For our susta...,2,2,-0.900904
1,"Verizon’s environmental, health and safety man...",3,3,-0.890918
2,"In 2019, the Company closed a series of transa...",2,2,-0.923925
3,"In December 2020, the AUC approved the Electri...",2,2,-0.764115
4,"Finally, there is a reputational risk linked t...",2,2,-0.956473
...,...,...,...,...
392,Indirect emissions result from operational act...,1,1,-0.955989
393,"All data in this TCFD report is as of, or for ...",2,1,-0.870182
394,Outcome: The bank explained that it would be w...,1,2,-0.882809
395,"In 2020, Banco do Brasil Foundation celebrated...",2,2,-0.874456


In [40]:
import openai
import tiktoken
encoding = tiktoken.get_encoding("gpt2")
with open('c:\\data\\openAI-key-g.txt', 'r') as file:
    key = file.readline()
openai.api_key = key

# sava the key in the environment variable
import os
os.environ["OPENAI_API_KEY"] = key


In [46]:
import json
import time

def get_first_number(s):
    for i in range(len(s)):
        if s[i].isdigit():
            return int(s[i])
    # There is no number in the string, so search for the mapping_dict keys
    # find the if the some key from mappint_dict is in s and return the value, make the comarioson case insensitive
    for k in mappint_dict.keys():
        if k in s.lower():
            return mappint_dict[k]
    return -1

def batch_gpt_len(prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)
        
        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = get_first_number(r)
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print 
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys

In [ ]:
texts = df["text"].to_list()
prompt = '''
Classify the following climate related text in one of the four classes: {}'metrics':1, 'strategy': 2, 'risk': 3,'governance': 4}.
Answer only with the class number and nothing else. The text is: \n
'''
rez, rez_keys = batch_gpt_len(prompt, texts, 1)

In [47]:
texts = df["text"].to_list()
prompt = '''
Classify the following sentence in one of the four classes: ('metrics':1, 'strategy': 2, 'risk': 3,'governance': 4).
Answer only with the class number and noting else. The sentence is: \n
'''
rez, rez_keys = batch_gpt_len(prompt, texts, 1)

1 417 - 2 2: 2 875 - 3 3: 3 804 - 3 3: 4 1117 - 4 4: 5 607 - 3 3: 6 576 - 3 3: 7 573 - 3 3: 8 717 - 4 4: 9 492 - 3 3: 10 448 - 3 3: 
11 460 - 1 1: 12 414 - 3 3: 13 473 - 2 2: 14 634 - 3 3: 15 557 - 3 3: 16 388 - 3 3: 17 864 - 3 3: 18 623 - 4 4: 19 899 - governance (4) class. 4: 20 844 - Metrics 1: 
21 799 - 3 3: 22 550 - 3 3: 23 499 - 2 2: 24 439 - 1 1: 25 670 - 3 3: 26 802 - 3 3: 27 800 - 1 1: 28 397 - 3 3: 29 496 - 2 2: 30 691 - 3 3: 
31 855 - 2 2: 32 1025 - 4 4: 33 780 - 2 2: 34 522 - 4 4: 35 695 - 3 3: 36 512 - 2 2: 37 612 - 3 3: 38 490 - 3 3: 39 767 - 1 1: 40 646 - 2 2: 
41 438 - 2 2: 42 485 - 4 4: 43 745 - 3 3: 44 742 - 4 4: 45 456 - 1 1: 46 417 - 3 3: 47 406 - 1 1: 48 643 - 3 3: 49 512 - 3 3: 50 1335 - governance 4: 
51 646 - 2 2: 52 610 - 3 3: 53 422 - 4 4: 54 541 - 2 2: 55 567 - 1 1: 56 479 - 2 2: 57 807 - 2 2: 58 1580 - 3 (risk) 3: 59 405 - 4 4: 60 1166 - governance 4: 
61 443 - 4 4: 62 620 - 2 2: 63 724 - 3 3: 64 867 - 3 3: 65 1040 - 3 (risk) 3: 66 795 - 1 1: 67 752 - 3 3: 6

In [48]:
col_rez = "gpt_rez"
col_label = "gpt_label"
df[col_rez ] = rez
df[col_label] = rez_keys

C:\Users\dimit\AppData\Local\Temp\ipykernel_24720\3670326254.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_rez ] = rez
C:\Users\dimit\AppData\Local\Temp\ipykernel_24720\3670326254.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_label] = rez_keys


In [49]:
df.to_csv("chat_gpt_tcdf.csv")

In [51]:
df[col_label].unique()

array([2, 3, 4, 1], dtype=int64)

In [71]:
# select rows that have gpt_label not 1 or 0
df[df[col_label] != 1][df[col_label] != 0]

C:\Users\dimit\AppData\Local\Temp\ipykernel_30608\2609781461.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df[col_label] != 1][df[col_label] != 0]


,text,label,prediction,prediction_VALUE,gpt_rez,gpt_label,gpt_rez-s,gpt_label-s,gpt_rez-c,gpt_label-c,gpt_rez-e,gpt_label-e


In [68]:
# update the gpt_label-e to 0 where gpt_rez-e is No.
df.loc[df[col_rez] == "No.", col_label] = 0

In [52]:
# count the the different values in the gpt_label column
df[col_label].value_counts()


gpt_label
3    123
4     98
2     60
1     39
Name: count, dtype: int64

In [70]:
# replace the gpt_label wiht 1 whre the gpt_label is not -1
df.loc[df[col_label] == 2, col_label] = 1
df.loc[df[col_label] == -1, col_label] = 1

In [53]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support
sent_col = col_label
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.4569825209633571, 0.5080553209179025, 0.38449680573647527, None)
[[23  9  6 11]
 [15 49 83 50]
 [ 0  2 28 18]
 [ 1  0  6 19]]
              precision    recall  f1-score   support

           1       0.59      0.47      0.52        49
           2       0.82      0.25      0.38       197
           3       0.23      0.58      0.33        48
           4       0.19      0.73      0.31        26

    accuracy                           0.37       320
   macro avg       0.46      0.51      0.38       320
weighted avg       0.64      0.37      0.39       320



In [73]:
# create a new column with the gpt_label-all that is 1 if any of the gpt_label-e, gpt_label-s, gpt_label-c is 1
df["gpt_label-all"] = df["gpt_label-e"] + df["gpt_label-s"] + df["gpt_label-c"]
df.loc[df["gpt_label-all"] > 0, "gpt_label-all"] = 1


In [75]:
sent_col = "gpt_label-s"
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.7357046826282216, 0.821875, 0.7571223860296797, None)
[[ 67  13]
 [ 62 258]]
              precision    recall  f1-score   support

           0       0.52      0.84      0.64        80
           1       0.95      0.81      0.87       320

    accuracy                           0.81       400
   macro avg       0.74      0.82      0.76       400
weighted avg       0.87      0.81      0.83       400

